# Process INFORM Data Excel Files

This notebook processes all Excel files in the `inform_data` directory, extracts the "INFORM Severity - country" sheet from each, and combines them into CSV and parquet files.

In [17]:
import pandas as pd
import os
import re
from pathlib import Path

def extract_month_year(filename):
    """
    Extract month and year from filename in MONTH_YEAR format.
    Handles various filename patterns like:
    - 20221205_inform_severity_-_november_2022.xlsx -> november_2022
    - 202501_INFORM_Severity_-_January_2025.xlsx -> January_2025
    - 202511_inform_severity_-_late_november_2025_.xlsx -> november_2025
    """
    # Remove file extension
    name = filename.replace('.xlsx', '').replace('.XLSX', '')
    
    # Try to find pattern like "month_year" or "month_year_" at the end
    # Look for patterns like: word(s)_YYYY or YYYY_word(s)
    # Common pattern: something_month_year or something_YYYYMM
    
    # Pattern 1: Look for "month_year" pattern (e.g., "november_2022", "January_2025")
    # This matches: word(s) followed by underscore followed by 4-digit year
    pattern1 = r'([a-zA-Z]+(?:_[a-zA-Z]+)*)_(\d{4})(?:_|$)'
    match = re.search(pattern1, name, re.IGNORECASE)
    if match:
        month_part = match.group(1)
        year = match.group(2)
        # Convert month to lowercase for consistency, but keep original format
        month = month_part.lower()
        return month, year
    
    # Pattern 2: If filename starts with YYYYMM, extract from that
    # e.g., "202501_INFORM_Severity_-_January_2025" -> "January_2025"
    pattern2 = r'(\d{6})_.*?([a-zA-Z]+(?:_[a-zA-Z]+)*)_(\d{4})'
    match = re.search(pattern2, name, re.IGNORECASE)
    if match:
        month_part = match.group(2)
        year = match.group(3)
        month = month_part.lower()
        return month, year
    
    # Fallback: try to extract from any month_year pattern
    pattern3 = r'([a-zA-Z]+)_(\d{4})'
    matches = re.findall(pattern3, name, re.IGNORECASE)
    if matches:
        # Take the last match (most likely to be the month_year we want)
        month, year = matches[-1]
        return month.lower(), year
    
    # If no pattern matches, return a placeholder
    return "unknown_date", "unknown_year"

In [18]:
# Get the directory path (adjust if running from a different location)
inform_data_dir = Path("inform_data")

# Get all Excel files
excel_files = list(inform_data_dir.glob("*.xlsx")) + list(inform_data_dir.glob("*.XLSX"))

if not excel_files:
    print("No Excel files found in inform_data directory")
else:
    print(f"Found {len(excel_files)} Excel files to process")

Found 64 Excel files to process


In [ ]:
dataframes = []

for excel_file in excel_files:
    try:
        print(f"Processing: {excel_file.name}")
        
        # Extract month_year from filename
        month, year = extract_month_year(excel_file.name)
        print(f"  Extracted date: {month} {year}")
        
        # Read the specific sheet
        # header=1 means use row 1 (second row, 0-indexed) as column names
        # This automatically skips row 0 (first row)
        df = pd.read_excel(excel_file, sheet_name="INFORM Severity - country", header=1)
        
        # Drop the first data row (which is row 2 in the original Excel file, the third row)
        # After using header=1, row 2 becomes index 0 in the dataframe
        if len(df) > 0:
            df = df.drop(df.index[0]).reset_index(drop=True)
            df = df.drop(df.index[0]).reset_index(drop=True)
        
        # Add month_year column
        df['month'] = month
        df['year'] = year
        
        dataframes.append(df)
        print(f"  Successfully loaded {len(df)} rows")
        
    except Exception as e:
        print(f"  Error processing {excel_file.name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

In [34]:
if not dataframes:
    print("No dataframes were successfully loaded")
else:
    # Concatenate all dataframes
    print("\nConcatenating all dataframes...")
    combined_df = pd.concat(dataframes, ignore_index=True)
    print(f"Total rows in combined dataframe: {len(combined_df)}")
    print(f"Total columns: {len(combined_df.columns)}")
    
    # Display the dataframe
    display(combined_df)


Concatenating all dataframes...
Total rows in combined dataframe: 5292
Total columns: 24


,CRISIS,CRISIS ID,COUNTRY,ISO3,DRIVERS,INFORM Severity Index,INFORM Severity category,INFORM Severity category.1,Trend (last 3 months),Reliability,...,People in need,Concentration of conditions,Complexity of the crisis,Society and safety,Operating environment,Regions,Last updated,month,year,TYPE OF CRISIS
0,Complex crisis in Afghanistan,AFG001,Afghanistan,AFG,"Conflict,Violence,Displacement,Drought,Earthqu...",4.5,5,Very High,Stable,Very High,...,5,4,4.3,4,4.5,Asia,2023-03-27 00:00:00,march,2023,NaN
1,Drought in South-West Angola,AGO002,Angola,AGO,Drought,3.1,4,High,Decreasing,High,...,3.7,4,2.2,3.1,1,Africa,2023-03-22 00:00:00,march,2023,NaN
2,Nagorno-Karabakh Conflict in Armenia,ARM002,Armenia,ARM,"Conflict,Displacement",1.1,2,Low,Decreasing,Medium,...,0.7,1,1.4,1.7,1,Middle east,2023-03-28 00:00:00,march,2023,NaN
3,Nagorno-Karabakh conflict in Azerbaijan,AZE002,Azerbaijan,AZE,"Conflict,Displacement",1.8,2,Low,Stable,Low,...,0,2,2.3,2.5,2,Middle east,2023-03-28 00:00:00,march,2023,NaN
4,Complex in Burundi,BDI001,Burundi,BDI,"Violence,Displacement,Floods",3.5,4,High,Stable,Very High,...,3.8,3,3.3,3,3.5,Africa,2023-03-17 00:00:00,march,2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5287,Russia-Ukraine conflict,UKR002,Ukraine,UKR,"Conflict,Displacement",4.4,5,Very High,Stable,Very High,...,5,4,4.1,2.4,5,Europe,2025-02-28 00:00:00,february,2025,NaN
5288,Complex crisis in Venezuela,VEN001,Venezuela,VEN,"Socio-political,Violence,Floods",4,4,High,Stable,Very High,...,4.8,3,3.7,3.3,4,Americas,2025-02-27 00:00:00,february,2025,NaN
5289,Conflict in Yemen,YEM001,Yemen,YEM,"Conflict,Displacement",4.5,5,Very High,Stable,Very High,...,5,4,4.5,3.8,5,Middle east,2025-02-10 00:00:00,february,2025,NaN
5290,Drought in Zambia,ZMB002,Zambia,ZMB,"Drought,Food Security",3.5,4,High,Decreasing,High,...,5,3,2.2,2.4,2,Africa,2025-02-27 00:00:00,february,2025,NaN


In [61]:
mask = ((df['DRIVERS'].apply(len) == 0) & (df['TYPE OF CRISIS'].notna()))
df[mask]

,CRISIS,CRISIS ID,COUNTRY,ISO3,DRIVERS,INFORM Severity Index,INFORM Severity category,INFORM Severity category.1,Trend (last 3 months),Reliability,...,People in need,Concentration of conditions,Complexity of the crisis,Society and safety,Operating environment,Regions,Last updated,month,year,TYPE OF CRISIS
82,Complex crisis in Afghanistan,AFG001,Afghanistan,AFG,[],4.6,5.0,Very High,Stable,High,...,5.0,4.0,4.3,4.0,4.5,Asia,2021-06-30,june,2021,Complex crisis
83,Nagorno-Karabakh Conflict in Armenia,ARM002,Armenia,ARM,[],1.7,2.0,Low,Stable,High,...,1.4,1.0,1.8,2.0,1.5,Middle east,2021-04-26,june,2021,Conflict
84,Nagorno-Karabakh conflict in Azerbaijan,AZE002,Azerbaijan,AZE,[],NaN,NaN,x,-,Low,...,NaN,NaN,3.0,2.9,3.0,Middle east,2021-03-14,june,2021,Conflict
85,Complex in Burundi,BDI001,Burundi,BDI,[],3.8,4.0,High,Increasing,High,...,4.0,4.0,3.3,3.1,3.5,Africa,2021-03-24,june,2021,Complex crisis
86,Conflict in Burkina Faso,BFA002,Burkina Faso,BFA,[],3.9,4.0,High,Stable,Very High,...,3.6,5.0,3.1,2.7,3.5,Africa,2021-06-30,june,2021,Conflict
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117,Floods in central Vietnam,VNM002,Vietnam,VNM,[],2.9,3.0,Medium,Increasing,High,...,3.3,3.0,2.5,3.3,1.5,Asia,2021-03-26,march,2021,Flood
5118,Cyclone Harold in Vanuatu,VUT002,Vanuatu,VUT,[],2.0,2.0,Low,Stable,Medium,...,1.8,3.0,1.1,1.1,1.0,Pacific,2020-07-02,march,2021,Tropical cyclone
5119,Conflict in Yemen,YEM001,Yemen,YEM,[],4.6,5.0,Very High,Stable,Medium,...,5.0,4.0,4.5,3.8,5.0,Middle east,2020-10-19,march,2021,Complex crisis
5120,Drought in Zambia,ZMB002,Zambia,ZMB,[],2.7,3.0,Medium,Stable,Medium,...,3.7,3.0,1.7,1.8,1.5,Africa,2020-12-17,march,2021,Drought


In [62]:
# Update DRIVERS with TYPE OF CRISIS when DRIVERS is empty, then remove TYPE OF CRISIS column
if 'DRIVERS' in combined_df.columns and 'TYPE OF CRISIS' in combined_df.columns:
    print("Updating empty DRIVERS with TYPE OF CRISIS values...")
    # Create mask for empty DRIVERS
    empty_drivers_mask = combined_df['DRIVERS'].apply(len) == 0
    
    # Update DRIVERS where it's empty: convert TYPE OF CRISIS to list format
    combined_df.loc[empty_drivers_mask, 'DRIVERS'] = combined_df.loc[empty_drivers_mask, 'TYPE OF CRISIS'].apply(
        lambda x: [x] if pd.notna(x) else []
    )
    
    # Remove TYPE OF CRISIS column
    combined_df = combined_df.drop(columns=['TYPE OF CRISIS'])
    print(f"Updated {empty_drivers_mask.sum()} rows and removed TYPE OF CRISIS column")

Updating empty DRIVERS with TYPE OF CRISIS values...
Updated 1892 rows and removed TYPE OF CRISIS column


In [63]:
# Save the updated dataframe to CSV and parquet
csv_path = Path("inform_severity_combined.csv")
parquet_path = Path("inform_severity_combined.parquet")

print(f"Saving updated CSV to: {csv_path}")
combined_df.to_csv(csv_path, index=False)
print(f"Successfully saved CSV file with {len(combined_df)} rows")

print(f"\nSaving updated parquet to: {parquet_path}")
combined_df.to_parquet(parquet_path, index=False)
print(f"Successfully saved parquet file with {len(combined_df)} rows")

print(f"\nColumns: {list(combined_df.columns)}")
print(f"\n✓ Complete! Files saved:")
print(f"  CSV: {csv_path}")
print(f"  Parquet: {parquet_path}")

Saving updated parquet to: inform_severity_combined.parquet
Successfully saved parquet file with 5292 rows
Columns: ['CRISIS', 'CRISIS ID', 'COUNTRY', 'ISO3', 'DRIVERS', 'INFORM Severity Index', 'INFORM Severity category', 'INFORM Severity category.1', 'Trend (last 3 months)', 'Reliability', 'Impact of the crisis', 'Geographical', 'Human', 'Conditions of people affected', 'People in need', 'Concentration of conditions', 'Complexity of the crisis', 'Society and safety', 'Operating environment', 'Regions', 'Last updated', 'month', 'year']


In [38]:
# Clean data types before saving to parquet
# Convert numeric columns that may have mixed types (strings like 'x' mixed with numbers)
print("Cleaning data types for parquet compatibility...")
numeric_columns = ['INFORM Severity Index', 'INFORM Severity category', 
                'Conditions of people affected', 'People in need',
                   'Concentration of conditions', 'Complexity of the crisis', 
                   'Society and safety', 'Operating environment', 'Impact of the crisis']

# Date columns that should be converted to datetime
date_columns = ['Last updated']

# Convert DRIVERS from comma-separated string to list
if 'DRIVERS' in combined_df.columns:
    print("Converting DRIVERS from comma-separated to list format...")
    combined_df['DRIVERS'] = combined_df['DRIVERS'].apply(
        lambda x: [item.strip() for item in str(x).split(',')] if pd.notna(x) and str(x).strip() else []
    )

# Convert date columns first
for col in date_columns:
    if col in combined_df.columns:
        # Convert to datetime, replacing invalid dates with NaT
        combined_df[col] = pd.to_datetime(combined_df[col], errors='coerce')

# Also check for any object-type columns that might be numeric (but skip date columns)
for col in combined_df.columns:
    if col not in numeric_columns and col not in date_columns and combined_df[col].dtype == 'object':
        # Try to convert to numeric - if it works, it's likely a numeric column
        try:
            # Check if column contains mostly numeric values (excluding NaN)
            non_null = combined_df[col].dropna()
            if len(non_null) > 0:
                # Try converting a sample
                sample = non_null.head(100)
                numeric_count = pd.to_numeric(sample, errors='coerce').notna().sum()
                # If more than 50% can be converted to numeric, treat as numeric column
                if numeric_count / len(sample) > 0.5:
                    numeric_columns.append(col)
        except:
            pass

for col in numeric_columns:
    if col in combined_df.columns:
        # Convert to numeric, replacing non-numeric values with NaN
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

print(f"Cleaned {len(numeric_columns)} numeric columns")
print(f"Cleaned {len(date_columns)} date columns")
print("Data cleaning complete!")

Cleaning data types for parquet compatibility...
Converting DRIVERS from comma-separated to list format...
Cleaned 9 numeric columns
Cleaned 1 date columns
Data cleaning complete!


In [39]:
combined_df

,CRISIS,CRISIS ID,COUNTRY,ISO3,DRIVERS,INFORM Severity Index,INFORM Severity category,INFORM Severity category.1,Trend (last 3 months),Reliability,...,People in need,Concentration of conditions,Complexity of the crisis,Society and safety,Operating environment,Regions,Last updated,month,year,TYPE OF CRISIS
0,Complex crisis in Afghanistan,AFG001,Afghanistan,AFG,"[Conflict, Violence, Displacement, Drought, Ea...",4.5,5.0,Very High,Stable,Very High,...,5.0,4.0,4.3,4.0,4.5,Asia,2023-03-27,march,2023,NaN
1,Drought in South-West Angola,AGO002,Angola,AGO,[Drought],3.1,4.0,High,Decreasing,High,...,3.7,4.0,2.2,3.1,1.0,Africa,2023-03-22,march,2023,NaN
2,Nagorno-Karabakh Conflict in Armenia,ARM002,Armenia,ARM,"[Conflict, Displacement]",1.1,2.0,Low,Decreasing,Medium,...,0.7,1.0,1.4,1.7,1.0,Middle east,2023-03-28,march,2023,NaN
3,Nagorno-Karabakh conflict in Azerbaijan,AZE002,Azerbaijan,AZE,"[Conflict, Displacement]",1.8,2.0,Low,Stable,Low,...,0.0,2.0,2.3,2.5,2.0,Middle east,2023-03-28,march,2023,NaN
4,Complex in Burundi,BDI001,Burundi,BDI,"[Violence, Displacement, Floods]",3.5,4.0,High,Stable,Very High,...,3.8,3.0,3.3,3.0,3.5,Africa,2023-03-17,march,2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5287,Russia-Ukraine conflict,UKR002,Ukraine,UKR,"[Conflict, Displacement]",4.4,5.0,Very High,Stable,Very High,...,5.0,4.0,4.1,2.4,5.0,Europe,2025-02-28,february,2025,NaN
5288,Complex crisis in Venezuela,VEN001,Venezuela,VEN,"[Socio-political, Violence, Floods]",4.0,4.0,High,Stable,Very High,...,4.8,3.0,3.7,3.3,4.0,Americas,2025-02-27,february,2025,NaN
5289,Conflict in Yemen,YEM001,Yemen,YEM,"[Conflict, Displacement]",4.5,5.0,Very High,Stable,Very High,...,5.0,4.0,4.5,3.8,5.0,Middle east,2025-02-10,february,2025,NaN
5290,Drought in Zambia,ZMB002,Zambia,ZMB,"[Drought, Food Security]",3.5,4.0,High,Decreasing,High,...,5.0,3.0,2.2,2.4,2.0,Africa,2025-02-27,february,2025,NaN


In [40]:
# Save to CSV file
csv_path = Path("inform_severity_combined.csv")
print(f"Saving CSV to: {csv_path}")
combined_df.to_csv(csv_path, index=False)
print(f"Successfully saved CSV file with {len(combined_df)} rows")

# Save to parquet file
parquet_path = Path("inform_severity_combined.parquet")
print(f"\nSaving parquet to: {parquet_path}")
combined_df.to_parquet(parquet_path, index=False)
print(f"Successfully saved parquet file with {len(combined_df)} rows")

print(f"\n✓ Complete! Files saved:")
print(f"  CSV: {csv_path}")
print(f"  Parquet: {parquet_path}")

Saving CSV to: inform_severity_combined.csv
Successfully saved CSV file with 5292 rows

Saving parquet to: inform_severity_combined.parquet
Successfully saved parquet file with 5292 rows

✓ Complete! Files saved:
  CSV: inform_severity_combined.csv
  Parquet: inform_severity_combined.parquet


In [41]:
df = pd.read_parquet("inform_severity_combined.parquet")
df

,CRISIS,CRISIS ID,COUNTRY,ISO3,DRIVERS,INFORM Severity Index,INFORM Severity category,INFORM Severity category.1,Trend (last 3 months),Reliability,...,People in need,Concentration of conditions,Complexity of the crisis,Society and safety,Operating environment,Regions,Last updated,month,year,TYPE OF CRISIS
0,Complex crisis in Afghanistan,AFG001,Afghanistan,AFG,"[Conflict, Violence, Displacement, Drought, Ea...",4.5,5.0,Very High,Stable,Very High,...,5.0,4.0,4.3,4.0,4.5,Asia,2023-03-27,march,2023,NaN
1,Drought in South-West Angola,AGO002,Angola,AGO,[Drought],3.1,4.0,High,Decreasing,High,...,3.7,4.0,2.2,3.1,1.0,Africa,2023-03-22,march,2023,NaN
2,Nagorno-Karabakh Conflict in Armenia,ARM002,Armenia,ARM,"[Conflict, Displacement]",1.1,2.0,Low,Decreasing,Medium,...,0.7,1.0,1.4,1.7,1.0,Middle east,2023-03-28,march,2023,NaN
3,Nagorno-Karabakh conflict in Azerbaijan,AZE002,Azerbaijan,AZE,"[Conflict, Displacement]",1.8,2.0,Low,Stable,Low,...,0.0,2.0,2.3,2.5,2.0,Middle east,2023-03-28,march,2023,NaN
4,Complex in Burundi,BDI001,Burundi,BDI,"[Violence, Displacement, Floods]",3.5,4.0,High,Stable,Very High,...,3.8,3.0,3.3,3.0,3.5,Africa,2023-03-17,march,2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5287,Russia-Ukraine conflict,UKR002,Ukraine,UKR,"[Conflict, Displacement]",4.4,5.0,Very High,Stable,Very High,...,5.0,4.0,4.1,2.4,5.0,Europe,2025-02-28,february,2025,NaN
5288,Complex crisis in Venezuela,VEN001,Venezuela,VEN,"[Socio-political, Violence, Floods]",4.0,4.0,High,Stable,Very High,...,4.8,3.0,3.7,3.3,4.0,Americas,2025-02-27,february,2025,NaN
5289,Conflict in Yemen,YEM001,Yemen,YEM,"[Conflict, Displacement]",4.5,5.0,Very High,Stable,Very High,...,5.0,4.0,4.5,3.8,5.0,Middle east,2025-02-10,february,2025,NaN
5290,Drought in Zambia,ZMB002,Zambia,ZMB,"[Drought, Food Security]",3.5,4.0,High,Decreasing,High,...,5.0,3.0,2.2,2.4,2.0,Africa,2025-02-27,february,2025,NaN
